In [ ]:
from hdbcli import dbapi
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Connection database

In [ ]:
connection = dbapi.connect('34.254.159.229', 39015, 'SYSTEM', '')
connection.isconnected()

In [ ]:
cursor = connection.cursor()

# Import data

In [ ]:
# import data
t3n_text_index_df = pd.read_csv('data/T3NTEXTIND.csv')
t3n_df = pd.read_csv('data/T3N.csv')

In [ ]:
t3n_df = pd.read_csv('../praktikum2/t3n_data_cleaned.csv')

In [ ]:
t3n_df

# Teil 1

## 1. Ermitteln Sie die häufigsten Folgen von Adjektiv-Nomen Bigrammen im Corpus (Hinweis: wenn Sie mit SQL arbeiten, können Sie einen non-equi auto-join verwenden)

In [ ]:
sql = 'select top 10 t1.TA_TOKEN, t2.TA_TOKEN, count(*) from "SYSTEM"."$TA_CDESCRIND" as t1, "SYSTEM"."$TA_CDESCRIND" as t2 where t1.cmplid=t2.cmplid and t1.TA_COUNTER=t2.TA_COUNTER-1 and t1.TA_SENTENCE=t2.TA_SENTENCE and t1.TA_TYPE=\'adjective\' and t2.TA_TYPE=\'noun\' group by t1.TA_TOKEN, t2.TA_TOKEN order by count(*) desc'
cursor.execute(sql)
adjective_noun = cursor.fetchall()
adjective_noun_df = pd.DataFrame(adjective_noun)
adjective_noun_df.columns = ['adjective', 'noun', 'count']
adjective_noun_df

## 2. Ermitteln Sie die häufigsten Co-occurrences von Adjektiven innerhalb eines Satzes (Position egal).

In [ ]:
t3n_adjectives = t3n_df[t3n_df['TA_TYPE']=='adjective']

In [ ]:
grouped_coocurrencies = t3n_adjectives.groupby(['ID','TA_SENTENCE'])['TA_TOKEN_LOW']

In [ ]:
coocurrencies = {}

#generate "matrix" with coocurrencies. we count the coocurrencies of each adverb per sentence. 
for name, group in grouped_coocurrencies:
    for index_i, token_i in group.iteritems():
        type(token_i)
        if token_i not in coocurrencies:
            coocurrencies[token_i] = {}
            
        for index_j, token_j in group.iteritems():
            if token_j not in coocurrencies[token_i]:
                coocurrencies[token_i][token_j] = 0
            else:
                coocurrencies[token_i][token_j]+=1 

In [ ]:
ordered_coocurrencies = {}

for coocurrency in coocurrencies:
    coocurrencies_for_this_token = coocurrencies[coocurrency]
    for coocurrency_for_this_token in coocurrencies_for_this_token:
        if(coocurrency != coocurrency_for_this_token):
            ordered_coocurrencies[coocurrency + '-' + coocurrency_for_this_token] = coocurrencies_for_this_token[coocurrency_for_this_token] 
            
ordered_coocurrencies = {k: v for k, v in sorted(ordered_coocurrencies.items(), key=lambda item: item[1],reverse=True)}        
    

In [ ]:
def drop_dublicates(ordered_coocurrencies):
    for occurency in ordered_coocurrencies.copy():
        split_occurency = occurency.split('-')
        dublicate_occurency = split_occurency[1] + '-' + split_occurency[0]
        if dublicate_occurency in ordered_coocurrencies and occurency in ordered_coocurrencies:
            ordered_coocurrencies.pop(dublicate_occurency)
    return ordered_coocurrencies

In [ ]:
drop_dublicates(ordered_coocurrencies)

## 3. Berechnen Sie die top-3 tf-idf (ntn nach der SMART Notation) Werte von Nomen im Corpus. Schauen Sie sich die entsprechenden Dokumente und die jeweiligen tf bzw. idf Wertkomponenten an.

### tf

In [ ]:
# create SQL-View MAX_FREQ_NOUN
sql = 'create view MAX_FREQ_NOUN as select top 1 t1.TA_TOKEN as noun, count(*) as maxfreq from "SYSTEM"."$TA_CDESCRIND" as t1 where t1.TA_TYPE=\'noun\' group by t1.TA_TOKEN order by count(*) desc'
cursor.execute(sql)

In [ ]:
# drop SQL-View MAX_FREQ_NOUN
sql_drop_view = 'drop view MAX_FREQ_NOUN'
cursor.execute(sql_drop_view)

In [ ]:
# tf
sql = 'select top 3 t1.TA_TOKEN, (count(*)/t2.maxfreq) from "SYSTEM"."$TA_CDESCRIND" as t1, "SYSTEM"."MAX_FREQ_NOUN" as t2 where t1.TA_TYPE=\'noun\' group by t1.TA_TOKEN, t2.maxfreq order by count(*) desc'
cursor.execute(sql)
tf = cursor.fetchall()
tf_df = pd.DataFrame(tf)
tf_df.columns = ['noun', 'tf']
tf_df

### idf

In [ ]:
# create SQL-View MAX_COUNT_NOUNS
sql = 'create view MAX_COUNT_NOUNS as select top 3 t1.TA_TOKEN as noun from "SYSTEM"."$TA_CDESCRIND" as t1 where t1.TA_TYPE=\'noun\' group by t1.TA_TOKEN order by count(*) desc'
cursor.execute(sql)

In [ ]:
# create SQL-View COUNT_DOCS
sql = 'create view COUNT_DOCS as select count(DISTINCT cmplid) as n_total from "SYSTEM"."$TA_CDESCRIND"'
cursor.execute(sql)

In [ ]:
# create SQL-View VALUES_IDF
sql = 'create view VALUES_IDF as select t1.TA_TOKEN, count(DISTINCT cmplid) as count_docs_term, t3.n_total from "SYSTEM"."$TA_CDESCRIND" as t1, MAX_COUNT_NOUNS as t2, COUNT_DOCS as t3 where TA_TOKEN=t2.noun group by t1.TA_TOKEN, t3.n_total'
cursor.execute(sql)

In [ ]:
# drop SQL-Views
sql_drop_view = 'drop view <view_name>'
cursor.execute(sql_drop_view)

In [ ]:
# idf
sql = 'select TA_TOKEN, count_docs_term, n_total, LOG(10,(n_total/count_docs_term)) as idf from VALUES_IDF'
cursor.execute(sql)
idf = cursor.fetchall()
idf_df = pd.DataFrame(idf)
idf_df.columns = ['Token', 'count_docs_term', 'n_total', 'idf']
idf_df

## 4. Berücksichtigen Sie nur Adjektiv-Nomen Bigramme in Dokumenten mit der ID<1000000. Ermitteln Sie anhand des Chi2 Tests die 3 statistisch signifikantesten und die 3 am wenigsten signifikant zusammenhängenden Bigramme mit w1=* (beliebig) und w2=Tire.

In [ ]:
# create view
# all adjective-noun bigrams with ID < 1000000 and w1=* and w2=Tire
sql = 'create view adjective_noun_bigram as select t1.TA_TOKEN as adjective, t2.TA_TOKEN as noun, count(*) as  w1_and_w2 from "SYSTEM"."$TA_CDESCRIND" as t1, "SYSTEM"."$TA_CDESCRIND" as t2 where t1.cmplid=t2.cmplid and t1.TA_COUNTER=t2.TA_COUNTER-1 and t1.TA_SENTENCE=t2.TA_SENTENCE and t1.TA_TYPE=\'adjective\' and t2.TA_TYPE=\'noun\' and t1.cmplid<1000000 and t2.TA_TOKEN=\'TIRE\' group by t1.TA_TOKEN, t2.TA_TOKEN order by count(*) desc'
cursor.execute(sql)

In [ ]:
# create view
# all adjective-noun bigrams with ID < 1000000 and w1=* and w2=not_Tire
sql = 'create view adjective_noun_bigram_not_tire as select t1.TA_TOKEN as adjective, t2.TA_TOKEN as noun, count(*) as  w1_and_not_w2 from "SYSTEM"."$TA_CDESCRIND" as t1, "SYSTEM"."$TA_CDESCRIND" as t2 where t1.cmplid=t2.cmplid and t1.TA_COUNTER=t2.TA_COUNTER-1 and t1.TA_SENTENCE=t2.TA_SENTENCE and t1.TA_TYPE=\'adjective\' and t2.TA_TYPE=\'noun\' and t1.cmplid<1000000 and t2.TA_TOKEN!=\'TIRE\' group by t1.TA_TOKEN, t2.TA_TOKEN order by count(*) desc'
cursor.execute(sql)

In [ ]:
# drop SQL-Views
sql_drop_view = 'drop view <view_name>'
cursor.execute(sql_drop_view)

In [ ]:
cursor.execute("""
select adjective, noun, o11, o12, o21, o22 from 
(
select t1.adjective, t1.noun, w1_and_w2 as o11, (sum_w1_and_w2-SUM(w1_and_w2)) as o12, t3.sum_w1_and_not_w2 as o21, (sum_NOT_w2 - t3.sum_w1_and_not_w2) as o22 from 
"SYSTEM"."ADJECTIVE_NOUN_BIGRAM" as t1, 
(select SUM(w1_and_w2) as sum_w1_and_w2 from "SYSTEM"."ADJECTIVE_NOUN_BIGRAM") as t2,
(select adjective, SUM(w1_and_not_w2) as sum_w1_and_not_w2 from "SYSTEM"."ADJECTIVE_NOUN_BIGRAM_NOT_TIRE" group by adjective) as t3,
(select SUM(w1_and_not_w2) as sum_NOT_w2 from "SYSTEM"."ADJECTIVE_NOUN_BIGRAM_NOT_TIRE" order by sum_NOT_w2 desc) as t4 where t3.adjective=t1.adjective group by t1.adjective, t1.noun, w1_and_w2, t2.sum_w1_and_w2, t3.sum_w1_and_not_w2, t4.sum_NOT_w2
)
""");

In [ ]:
values_list = []
header = ["adjective", "noun", "o11", "o12", "o21", "o22", "chi_square"];
for item in data:
    o11=item[2]
    o12=item[3]
    o21= item[4]
    o22=item[5]
    chi_square = ((102848 * (o11 * o22 - o12 * o21)**2) / ((o11 + o12) * (o11 + o21) * (o12 + o22) * (o21 + o22)))
    values_list.append((item[0], item[1], item[2], item[3], item[4], item[5], chi_square))

values_list.sort(key=lambda x: x[6], reverse=True);
df = pd.DataFrame(values_list)
df.columns = [header]
df

## 5. Implementieren Sie eine Ähnlichkeitsanfrage, die für einen gegebenen Anfragevektor (egal ob existierendes Dokument oder eigene Suchanfrage) die gerankten top n Dokumente anhand der Wortvektoren (nur Nomen ohne IDF-Gewichtung!) zurückgibt mit Hilfe von 
## a. Skalarprodukt (nnn nach SMART Notation)


## b. Kosinus Ähnlichkeit (nnc nach SMART Notation)

## 6. Welches sind die jeweils ähnlichsten Dokumente für Complaint 119408? Wie unterscheiden sich die Ergebnisse grundsätzlich?

## 7. Testen Sie die Ähnlichkeitsanfragen für Ihren eigenen Datensatz

# Teil 2
## 1. Implementieren oder wrappen Sie eine Python-Funktion (gerne mit Hilfe üblicher Bibliotheken wie sklearn.metrics.precision_recall_curve), die für Ihre vorbereiteten Anfragevektoren und die mit der Kosinus-Ähnlichkeit ermittelten Dokumente die Precision-Recall Plots zeichnet (Interpolation ist nicht nötig).

## 2. Wie interpretieren Sie die Retrieval-Ergebnisse und die Plots (für Ihre eigenen Anfragen und mindestens für die Anfrage {'AIR': 2, 'BAG': 2, 'DEPLOYMENT':2, 'ACCIDENT':2} an NHTSA?

# Teil 3
## 1. Führen Sie das Python Skript runMinHashExample aus. Versuchen Sie anhand des Tutorials und des Codes nachzuvollziehen, was in den einzelnen Schritten passiert.


## 2. Die Ergebnisse sind noch nicht ganz ideal (falls zufällig doch, probieren Sie andere Random Seeds aus. Überlegen Sie, welche (einfachen!) Freiheitsgrade Sie haben, um diese zu verbessern. Setzen Sie diese um und prüfen Sie jeweils, ob Sie eine Verbesserung erzielen.

## 3. Implementieren Sie als Alternative zum wortbasierten Shingling, ein zeichenbasiertes Shingling und vergleichen Sie die Ergebnisse systematisch für mehrere Fenstergrößen (und ggf. anderer variierender Freiheitsgrade).